<h2>This notebook when run will produce a module which predicts the chance of death of patients in ICU after 24 hours</h2>

Importing libraries

In [1]:
import time

# visualizatoin 
import matplotlib.pyplot as plt 

# data wrangling
import pandas as pd
import numpy as np 

# data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import SimpleImputer, KNNImputer

from fairlearn.reductions import *
from fairlearn.metrics import *



#learning
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
import lightgbm as lgb


#post-processing
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

Loading data

In [2]:
# description
description = pd.read_csv('data/WiDS_Datathon_2020_Dictionary.csv')
description_dict = description.set_index('Variable Name').to_dict(orient='index')
# data
df = pd.read_csv('data/training_v2.csv')

df.head()

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,...,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,66154,25312,118,0,68.0,22.73,0,Caucasian,M,180.3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,114252,59342,81,0,77.0,27.42,0,Caucasian,F,160.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory
2,119783,50777,118,0,25.0,31.95,0,Caucasian,F,172.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic
3,79267,46918,118,0,81.0,22.64,1,Caucasian,F,165.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular
4,92056,34377,33,0,19.0,NaN,0,Caucasian,M,188.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma


<h3>Pre-processing</h3>

In [3]:
test_size = 0.2 # proportion for train versus test+val split
val_size = 0.5 # proportion for test versus valsplit
random_state = 42  # random state is used to set a seed for randomness, which is only relevant for reproducibility purposes
max_missing = 0.8  # maximum percentage of missing values for a column to be dropped

In [4]:
start_time = time.time()

# save features
X = df.copy().drop(['hospital_death', 'patient_id', 'encounter_id', 'hospital_id', 'icu_id', # drop identifiers
                    'apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob', # drop APACHE scores
                    'apache_2_bodysystem', 'bmi'], # drop because of similarity with apache_3j_bodysystem
                   axis=1)
# save target variable
y = df['hospital_death'].copy()
# save APACHE scores for later evaluation on train / test / validation data
y_apache = df['apache_4a_hospital_death_prob'].copy()

""" SPLIT DATA SET """
# split the dataset into train and test+validation set
(
    X_train,
    X_test,
    y_train,
    y_test,
    y_apache_train,
    y_apache_test,
    ) = train_test_split(X, y, y_apache, 
                         test_size=test_size, # used for testing and validation
                         random_state=random_state # for reproducibility
                        ) 
# split the test set into test + validation set
(
    X_val,
    X_test,
    y_val,
    y_test,
    y_apache_val,
    y_apache_test,
    ) = train_test_split(X_test, y_test, y_apache_test, 
                         test_size=val_size, # used for testing and validation
                         random_state=random_state # for reproducibility
                        ) 

# """MISSING VALUES"""
# # drop columns with many missing values
# missing = X_train.isna().sum() > max_missing * len(X_train)
# missing = missing[missing].index
# X_train = X_train.drop(missing, axis=1)
# X_val = X_val.drop(missing, axis=1)
# X_test = X_test.drop(missing, axis=1)

"""FURTHER PROCESSING PIPELINE"""
# define pre-processing steps for numerical features
num_transformer = Pipeline(steps=[("constant", VarianceThreshold()), # remove constant features
                                  ("imputer", SimpleImputer(strategy="median")),
                                 ])
# define preprocessing steps for categorical features
cat_transformer = Pipeline(steps=[("encoder", OneHotEncoder(drop='first', sparse=False, handle_unknown="ignore"))])
# create preprocessing pipeline
prep_pipeline = ColumnTransformer(
    transformers=[
        ('num', num_transformer, make_column_selector(dtype_exclude=object)), # apply to columns NOT of type object (int or float)
        ('cat', cat_transformer, make_column_selector(dtype_include=object)) # apply to columns of type object
    ])
# pipeline
prep_pipeline.fit(X_train, y_train)
display(prep_pipeline) # disply preprocessing pipeline

# transform data sets
X_train = pd.DataFrame(prep_pipeline.transform(X_train), columns=prep_pipeline.get_feature_names_out())
X_val = pd.DataFrame(prep_pipeline.transform(X_val), columns=prep_pipeline.get_feature_names_out())
X_test = pd.DataFrame(prep_pipeline.transform(X_test), columns=prep_pipeline.get_feature_names_out())
        
"""PRINT STATS"""
print("Time: %.2fs" % (time.time() - start_time))
print("Train set: %s rows, %s columns" % X_train.shape)
print("Validation set: %s rows, %s columns" % X_val.shape)
print("Test set: %s rows, %s columns" % X_test.shape)

/home/aertbei/Repos/2IX30_17/venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('constant',
                                                  VarianceThreshold()),
                                                 ('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f451c316aa0>),
                                ('cat',
                                 Pipeline(steps=[('encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f451c316a10>)])

Time: 5.85s
Train set: 73370 rows, 217 columns
Validation set: 9171 rows, 217 columns
Test set: 9172 rows, 217 columns


Boruta implementation based on original code from (source here)

#checking feature importance from boruta
X_boruta = X_train.values
Y_boruta = y_train.values

# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=10)
# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)
# find all relevant features - 5 features should be selected
feat_selector.fit(X_boruta, Y_boruta)
# check selected features - first 5 features are selected
feat_selector.support_
# check ranking of features
feat_selector.ranking_
# call transform() on X to filter it down to selected features
X_filtered = feat_selector.transform(X_boruta)

Adding a cross-validation method (Modified from [author here])

In [5]:
# K-Fold Cross-Validation
#taken from https://www.section.io/engineering-education/how-to-implement-k-fold-cross-validation/
from sklearn.model_selection import cross_validate

def cross_validation(model, _X, _y, _cv=5):
      '''Function to perform 5 Folds Cross-Validation
       Parameters
       ----------
      model: Python Class, default=None
              This is the machine learning algorithm to be used for training.
      _X: array
           This is the matrix of features.
      _y: array
           This is the target variable.
      _cv: int, default=5
          Determines the number of folds for cross-validation.
       Returns
       -------
       The function returns a dictionary containing the metrics 'accuracy', 'precision',
       'recall', 'f1' for both training set and validation set.
      '''
      _scoring = ['accuracy', 'roc_auc']
      results = cross_validate(estimator=model,
                               X=_X,
                               y=_y,
                               cv=_cv,
                               scoring=_scoring,
                               return_train_score=True)
      
      return {"Training Accuracy scores": results['train_accuracy'],
              "Mean Training Accuracy": results['train_accuracy'].mean()*100,
              "Training AUC scores": results['train_roc_auc'],
              "Mean Training AUC score": results['train_roc_auc'].mean(),
              "Validation Accuracy scores": results['test_accuracy'],
              "Mean Validation Accuracy": results['test_accuracy'].mean()*100,
              "Validation AUC Scores": results['test_roc_auc'],
              "Mean Validation AUC score": results['test_roc_auc'].mean()
              }

Implementation of the model

In [6]:
borutacolumns = ['num__age', 'num__elective_surgery', 'num__pre_icu_los_days',
       'num__weight', 'num__albumin_apache', 'num__apache_2_diagnosis',
       'num__apache_3j_diagnosis', 'num__apache_post_operative',
       'num__bilirubin_apache', 'num__bun_apache',
       'num__creatinine_apache', 'num__fio2_apache',
       'num__gcs_eyes_apache', 'num__gcs_motor_apache',
       'num__gcs_verbal_apache', 'num__glucose_apache',
       'num__heart_rate_apache', 'num__hematocrit_apache',
       'num__intubated_apache', 'num__map_apache', 'num__paco2_apache',
       'num__paco2_for_ph_apache', 'num__pao2_apache', 'num__ph_apache',
       'num__resprate_apache', 'num__temp_apache',
       'num__urineoutput_apache', 'num__ventilated_apache',
       'num__wbc_apache', 'num__d1_diasbp_min',
       'num__d1_diasbp_noninvasive_min', 'num__d1_heartrate_max',
       'num__d1_heartrate_min', 'num__d1_mbp_invasive_min',
       'num__d1_mbp_max', 'num__d1_mbp_min',
       'num__d1_mbp_noninvasive_max', 'num__d1_mbp_noninvasive_min',
       'num__d1_resprate_max', 'num__d1_resprate_min', 'num__d1_spo2_min',
       'num__d1_sysbp_invasive_min', 'num__d1_sysbp_max',
       'num__d1_sysbp_min', 'num__d1_sysbp_noninvasive_max',
       'num__d1_sysbp_noninvasive_min', 'num__d1_temp_max',
       'num__d1_temp_min', 'num__h1_diasbp_min',
       'num__h1_diasbp_noninvasive_min', 'num__h1_heartrate_max',
       'num__h1_heartrate_min', 'num__h1_mbp_min',
       'num__h1_mbp_noninvasive_min', 'num__h1_resprate_max',
       'num__h1_resprate_min', 'num__h1_spo2_min', 'num__h1_sysbp_max',
       'num__h1_sysbp_min', 'num__h1_sysbp_noninvasive_max',
       'num__h1_sysbp_noninvasive_min', 'num__h1_temp_max',
       'num__h1_temp_min', 'num__d1_albumin_max', 'num__d1_albumin_min',
       'num__d1_bilirubin_max', 'num__d1_bilirubin_min',
       'num__d1_bun_max', 'num__d1_bun_min', 'num__d1_calcium_min',
       'num__d1_creatinine_max', 'num__d1_creatinine_min',
       'num__d1_glucose_max', 'num__d1_glucose_min', 'num__d1_hco3_max',
       'num__d1_hco3_min', 'num__d1_hemaglobin_max',
       'num__d1_hemaglobin_min', 'num__d1_hematocrit_max',
       'num__d1_hematocrit_min', 'num__d1_inr_max', 'num__d1_inr_min',
       'num__d1_lactate_max', 'num__d1_lactate_min',
       'num__d1_platelets_max', 'num__d1_platelets_min',
       'num__d1_sodium_max', 'num__d1_wbc_max', 'num__d1_wbc_min',
       'num__h1_glucose_min', 'num__h1_inr_max', 'num__h1_inr_min',
       'num__d1_arterial_pco2_max', 'num__d1_arterial_pco2_min',
       'num__d1_arterial_ph_max', 'num__d1_arterial_ph_min',
       'num__d1_arterial_po2_max', 'num__d1_arterial_po2_min',
       'num__d1_pao2fio2ratio_max', 'num__d1_pao2fio2ratio_min',
       'cat__hospital_admit_source_Operating Room',
       'cat__icu_admit_source_Operating Room / Recovery',
       'cat__apache_3j_bodysystem_Metabolic']

In [7]:
# X_train_df = pd.DataFrame(X_train, columns=X_train.columns[borutacolumns]) #feat_selector..support_
# X_val_df = pd.DataFrame(X_val, columns=X_val.columns[borutacolumns])

X_train_df = X_train[borutacolumns]
X_val_df = X_val[borutacolumns]

In [8]:
train_data = lgb.Dataset(X_train_df)
val_data = lgb.Dataset(X_val)

In [19]:
param = {'num_leaves': 100, 'objective': 'binary'}
param['metric'] = 'auc'

In [20]:
constraint = ErrorRateParity(difference_bound=0.1)

In [21]:
sensitive_features = X_train_df[["num__age"]]

In [22]:
#a gradient based tree with 100 leaves 
modelLight = lgb.LGBMClassifier(num_leaves=500, class_weight="balanced")
reduction = ExponentiatedGradient(modelLight, constraint)
reduction.fit(X_train_df, y_train, sensitive_features=sensitive_features)

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.ErrorRateParity object at 0x7f450afa3af0>,
                      estimator=LGBMClassifier(class_weight='balanced',
                                               num_leaves=500),
                      nu=1.180337467640127e-05)

In [ ]:
#cross-val with sklearn api for gradient based
cross_validation(reduction, X_train_df, y_train)

In [ ]:

from sklearn.calibration import CalibratedClassifierCV

modelLightCal = CalibratedClassifierCV(reduction)
modelLightCal.fit(X_train_df, y_train)


Post-processing basic explainations only for 1 run of CV

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

predicted= reduction.predict(X_val_df)

conf_mat = confusion_matrix(y_val, predicted)

disp = ConfusionMatrixDisplay(conf_mat)
disp.plot()

In [ ]:
reduction.predict_proba(X_val_df)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from sklearn.calibration import CalibratedClassifierCV, CalibrationDisplay

clf_list = [
    (reduction, "LightGBM"),
]

fig = plt.figure(figsize=(10, 10))
gs = GridSpec(4, 2)
colors = plt.cm.get_cmap("Dark2")

ax_calibration_curve = fig.add_subplot(gs[:2, :2])
calibration_displays = {}
for i, (clf, name) in enumerate(clf_list):
    clf.fit(X_train_df, y_train)
    display = CalibrationDisplay.from_estimator(
        clf,
        X_val_df,
        y_val,
        n_bins=10,
        name=name,
        ax=ax_calibration_curve,
        color=colors(i),
    )
    calibration_displays[name] = display

ax_calibration_curve.grid()
ax_calibration_curve.set_title("Calibration plots")

# Add histogram
grid_positions = [(2, 0), (2, 1), (3, 0), (3, 1)]
for i, (_, name) in enumerate(clf_list):
    row, col = grid_positions[i]
    ax = fig.add_subplot(gs[row, col])

    ax.hist(
        calibration_displays[name].y_prob,
        range=(0, 1),
        bins=10,
        label=name,
        color=colors(i),
    )
    ax.set(title=name, xlabel="Mean predicted probability", ylabel="Count")

plt.tight_layout()
plt.show()


In [ ]:
#we create an empty dict to be used to store metrics
eval_res = {}


#this is one implementation of LightGBM used for post-process visualization, the actual model uses a sklearn API of LightGBM
bst = lgb.train(param, train_data, 100, valid_sets=[val_data, train_data], callbacks=[lgb.record_evaluation(eval_res)])

In [ ]:
lgb.plot_importance(bst)

In [ ]:

#this will not run unless graphviz is installed in the direction shown below
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz/bin/'

lgb.plot_tree(bst)

In [ ]:

lgb.plot_metric(booster=eval_res, metric='auc')